In [1]:
%matplotlib inline
import importlib
from pytorch_utils_oh_2 import *
import IPython

Pytorch utils oh: pytorch_utils_oh_2.py
Pytorch: 0.2.0_4


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [38]:
MODEL_SAVE_PATH = 'whole_gen_13_1_only_chars_and_attn'

In [4]:
import pytorch_utils_oh_2; importlib.reload(pytorch_utils_oh_2); from pytorch_utils_oh_2 import *;

Pytorch utils oh: pytorch_utils_oh_2.py
Pytorch: 0.2.0_4


# Data loading

In [5]:
# all_data = pickle.load(open("data/en_train_fixed_4_sentences.pkl", "rb" ))
all_data = pickle.load(open("data/en_train_fixed_5_manual.pkl", "rb" ))
# all_data_sentence_index = all_data.set_index('sentence_id')

In [6]:
sample_data = all_data.copy()
sample_data = sample_data[sample_data['class'] != 'NOT_CHANGED']
sample_data = sample_data[sample_data['class'] != 'MANUAL']
sample_data = pd.concat([sample_data, all_data[all_data['class_org'] == 'PLAIN'].sample(1000000)])
# sample_data = sample_data[sample_data['class'] != 'ELECTRONIC']
sample_data = sample_data.reset_index(drop=True)
print("Data rows: {},  (dropped rows: {})".format(len(sample_data), len(all_data)-len(sample_data)))
#del(all_data)

Data rows: 1654333,  (dropped rows: 8263859)


In [7]:
sample_data.sample(10)

,sentence_id,token_id,class,before,after,class_org,a_word_ind,sentence
1650614,527437,6,NOT_CHANGED,Heart,Heart,PLAIN,NaN,"soon after promoting "" this old <SAMPLE> of mi..."
948140,530535,3,NOT_CHANGED,and,and,PLAIN,NaN,the unusual shape <SAMPLE> diverse topography ...
806178,692652,0,NOT_CHANGED,Today,Today,PLAIN,NaN,"<SAMPLE> , 3 sr broadcasts as a relay of melbo..."
570253,651585,1,LETTERS,IC,i c,LETTERS,"[31, 21, 0]","rokos <SAMPLE> , larson dm , henry td et al ."
362657,418192,1,NUMBERS,2008,two thousand eight,DATE,"[5, 8, 16, 0]",in <SAMPLE> the state government finally accee...
520667,595422,4,NUMBERS,18 September 1945,the eighteenth of september nineteen forty five,DATE,"[11, 94, 12, 64, 7, 41, 14, 0]",rousy chanev ( born <SAMPLE> ) is a bulgarian ...
346553,399461,6,NUMBERS,1860,eighteen sixty,DATE,"[40, 39, 0]",it was described by thomson in <SAMPLE> .
1214426,586762,17,NOT_CHANGED,and,and,PLAIN,NaN,he was released in 1924 and became active as a...
50285,57857,2,NUMBERS,2011,twenty eleven,DATE,"[6, 48, 0]","records , <SAMPLE> ) pepper spray the tears aw..."
297848,343202,9,NUMBERS,1874,eighteen seventy four,DATE,"[40, 33, 19, 0]","ronge was born in vienna , austria , in <SAMPL..."


In [8]:
del(all_data)

In [9]:
categories_all = sorted(sample_data["class"].unique())
print(categories_all)
print(len(categories_all))
categories_index = dict((c, i) for i, c in enumerate(categories_all))

['ELECTRONIC', 'LETTERS', 'NOT_CHANGED', 'NUMBERS', 'PLAIN', 'VERBATIM']
6


In [78]:
sample_data.sample(2)

,sentence_id,token_id,class,before,after,class_org,a_word_ind,sentence
132260,152196,16,NUMBERS,6,six,CARDINAL,"[20, 0]",the highway passes over the norfolk southern r...
752067,580827,1,NOT_CHANGED,Bjork's,Bjork's,PLAIN,NaN,""" <SAMPLE> biophilia becomes first app in new ..."


In [76]:
sample_data = sample_data[sample_data['before'].str.len() > 0]

In [77]:
sample_data[sample_data['after'].str.len() < 1]

,sentence_id,token_id,class,before,after,class_org,a_word_ind,sentence


### Utils stuff

In [11]:
chars_normal, chars_normal_index = load_characters_pkl('data/en_features/chars_normal.pkl')
print(''.join(chars_normal))

<SOS><EOS>☒ !"#$%&'(),-./0123456789:;ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyz~£¥ª²³µº¼½¾éɒʻˈΩμ—€⅓⅔⅛


In [12]:
chars_after = sorted(list(set(''.join(list(sample_data['after'].unique())).lower())))
chars_after = [SOS_TOKEN, EOS_TOKEN, UNKNOWN_CHAR] + chars_after
chars_after_index = dict((c, i) for i, c in enumerate(chars_after))
print(''.join(chars_after))

<SOS><EOS>☒ '-abcdefghijklmnopqrstuvwxyzé


chars_after, chars_after_index = load_characters_pkl('data/en_features/chars_after_1.pkl')
print(''.join(chars_after))

In [13]:
wv_vecs, wv_words, wv_idx = load_glove('/home/ohu/koodi/data/glove_wordvec/glove.6B.50d.txt')

### After words handling

In [14]:
sos_tensor = torch.zeros(1, 1, len(chars_after_index))
sos_tensor[0, 0, chars_after_index[SOS_TOKEN]] = 1
sos_tensor.size()

torch.Size([1, 1, 33])

### More balanced sample

In [120]:
import collections

balanced_data_classes_select = list(sample_data.groupby('class'))

balanced_data_accessed_counter = 0 
balanced_data_randomize_freq = False
balanced_data_length = 0

last_samples = collections.deque(maxlen=100)
balanced_data_last_sample = last_samples

def balanced_data_randomize_org(max_len=30000):
    global balanced_data, balanced_data_length, balanced_data_accessed_counter, balanced_data_randomize_freq
    balanced_data = pd.concat([v.sample(min(max_len, len(v))) for k, v in balanced_data_classes_select])
    balanced_data = pd.concat([balanced_data, sample_data[sample_data['class']=='NOT_CHANGED'].sample(int(max_len*2))])
    balanced_data_length = len(balanced_data)
    balanced_data_randomize_freq = balanced_data_length * 0.2
    balanced_data_accessed_counter = 0
balanced_data_randomize = balanced_data_randomize_org

def balanced_data_sample_row():
    global balanced_data_accessed_counter
    global balanced_data_last_sample
    balanced_data_accessed_counter += 1
    if balanced_data_randomize_freq and balanced_data_accessed_counter > balanced_data_randomize_freq:
        balanced_data_randomize()
        
    sample = balanced_data.iloc[random.randint(1, balanced_data_length-1)]
    last_samples.append(sample)
    balanced_data_last_sample = sample
    return sample
    
balanced_data_randomize()

In [80]:
#all_data.groupby('class')['class'].count()
#sample_data.groupby('class')['class'].count()
balanced_data.groupby('class')['class'].count()

class
ELECTRONIC      4964
LETTERS        30000
NOT_CHANGED    90000
NUMBERS        30000
PLAIN          30000
VERBATIM       11741
Name: class, dtype: int64

In [124]:
balanced_data_sample_row()

sentence_id                                           365695
token_id                                                   0
class                                                  PLAIN
before                                                    dr
after                                                 doctor
class_org                                              PLAIN
a_word_ind                                          [113, 0]
sentence       <SAMPLE> guy liagre succeeded him from 2012 .
Name: 317119, dtype: object

In [126]:
last_samples[-2]

sentence_id                                               563382
token_id                                                       6
class                                                    LETTERS
before                                                        C-
after                                                          c
class_org                                                LETTERS
a_word_ind                                               [21, 0]
sentence       86 wing and its complement of <SAMPLE> 47 dako...
Name: 491851, dtype: object

### Samples

In [18]:
def get_random_sample():
    sample_row = balanced_data_sample_row()   
    return sample_row['before'], sample_row['after'].lower(), sample_row['class'], sample_row['sentence'].split(' ')
            
def tmp():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    print(s_class, ':', s_bef, '->', s_aft)
    print(s_sentence)
    print(string_to_tensor(s_bef, chars_normal_index).shape)
tmp()

PLAIN : vs -> versus
['"', 'ufc', '186', ':', 'johnson', '<SAMPLE>']
torch.Size([1, 3, 104])


In [19]:
%%time
get_random_sample()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 287 µs


('State', 'state', 'NOT_CHANGED', ['<SAMPLE>', 'of', 'alabama', '.'])

# Model functions

In [20]:
use_cuda = True

In [21]:
MAX_ATTENTION_LENGTH = 30

In [22]:
tmp = sample_data[sample_data['before'].str.len()>MAX_ATTENTION_LENGTH]
len(tmp)
tmp.sample(2)
# tmp[~tmp['before'].str.contains('/')].sample(2)

754

,sentence_id,token_id,class,before,after,class_org,a_word_ind,sentence
332252,382824,1,ELECTRONIC,http://www.qub.ac.uk/home/TheUniversity/Locati...,h t t p colon slash slash w w w dot q u b dot ...,ELECTRONIC,"[45, 30, 30, 24, 129, 101, 101, 52, 52, 52, 74...","see <SAMPLE> , for example ."
457102,524515,2,ELECTRONIC,http://brain.oxfordjournals.org/content/137/2/...,h t t p colon slash slash b r a i n dot o x f ...,ELECTRONIC,"[45, 30, 30, 24, 129, 101, 101, 36, 35, 22, 31...","online at <SAMPLE> , j . graham ."


### Encoder

In [25]:
class EncoderRNN(nn.Module):
    def __init__(self, words_input_size, chars_input_size, words_hidden_size, chars_hidden_size,
                 words_layers=1, chars_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.words_layers = words_layers
        self.chars_layers = chars_layers
        self.words_hidden_size = words_hidden_size
        self.chars_hidden_size = chars_hidden_size
        self.hidden_size = words_hidden_size + chars_hidden_size

        self.rnn_words = nn.GRU(words_input_size, words_hidden_size // 2, words_layers,
                                 batch_first=True, bidirectional=True)

        self.rnn_chars = nn.GRU(chars_input_size, chars_hidden_size // 2, chars_layers,
                                 batch_first=True, bidirectional=True)
        
    def forward(self, word_vectors, string_tensor, hidden = None, init_hidden = True):
        if init_hidden:
            hidden_words, hidden_chars = self.init_hidden()
        
        all_outputs_words, hidden_words = self.rnn_words(word_vectors, hidden_words)
        output_words = hidden_words.view(1, -1)
        
        all_outputs_chars, hidden_chars = self.rnn_chars(string_tensor, hidden_chars)
        output_chars = hidden_chars.view(1, -1)
        
        #hidden_states_cat = Variable(torch.zeros(MAX_ATTENTION_LENGTH, self.hidden_size)).cuda()
        #for ei in range(min(MAX_ATTENTION_LENGTH, len(string_tensor[0]))):
        #    hidden_states_cat[ei] = torch.cat((output_words, all_outputs_chars[0, ei].view(1,-1)), 1)

        all_outputs_chars_padded = Variable(torch.zeros(MAX_ATTENTION_LENGTH, self.chars_hidden_size)).cuda()
        att_length = min(len(all_outputs_chars[0]), MAX_ATTENTION_LENGTH-1)
        all_outputs_chars_padded[0:att_length] = all_outputs_chars[0][0:att_length]
        
        output = torch.cat((output_words, output_chars), 1)

        #output = torch.cat((output_words, output_chars), 1)
        
        #return output, all_outputs_chars
        return output, all_outputs_chars_padded

    def init_hidden(self):
        var1 = Variable(torch.zeros(2 * self.words_layers, 1, self.words_hidden_size // 2))
        var2 = Variable(torch.zeros(2 * self.chars_layers, 1, self.chars_hidden_size // 2))
        
        var1 = var1.cuda(); var2 = var2.cuda()
        return (var1, var2)
    
    
encoder_rnn = EncoderRNN(words_input_size=wv_vecs.shape[-1], chars_input_size=len(chars_normal),
                         words_hidden_size=128, chars_hidden_size=512,
                         words_layers=1, chars_layers=1).cuda()
encoder_rnn

EncoderRNN (
  (rnn_words): GRU(50, 64, batch_first=True, bidirectional=True)
  (rnn_chars): GRU(104, 256, batch_first=True, bidirectional=True)
)

In [27]:
def test_encoder_single_sample():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    #s_bef, s_aft, s_class, s_sentence = get_random_sample(True)
    print(s_bef)
    
    words_t = Variable(words_to_word_vectors_tensor(s_sentence, wv_vecs, wv_idx)).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    return encoder_rnn(words_t, string_t), s_bef
    
(tmp_encoder_output, tmp_encoder_outputs), tmp = test_encoder_single_sample()
tmp
tmp_encoder_output.size()
tmp_encoder_outputs.size()

died


'died'

torch.Size([1, 640])

torch.Size([30, 512])

### Decoder

In [28]:
class DecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, chars_encoded_size,
                 n_layers=1, dropout_p=0.1, max_length=MAX_ATTENTION_LENGTH):
        super(DecoderRNN, self).__init__()
        
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.max_length = max_length
        
        self.emb_lin = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.attn = nn.Linear(self.hidden_size+self.hidden_size, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size+chars_encoded_size, self.hidden_size)
        
        #self.module_attn = torch.nn.ModuleList([self.emb_lin, self.dropout, self.attn, self.attn_combine])
        
        
        self.rnn = nn.GRU(hidden_size, hidden_size, n_layers, batch_first=True, bidirectional=False)
        self.lin_out = nn.Linear(hidden_size, input_size)
        
        #self.module_rnn = torch.nn.ModuleList([self.rnn, self.lin_out])

    def forward(self, last_input, hidden, encoder_outputs):
        embedded = self.emb_lin(last_input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        embedded = embedded[0]
                
        attn_weights = F.softmax(self.attn(torch.cat((embedded, hidden), 1)))
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        #return embedded, attn_applied
        
        rnn_input = torch.cat((embedded, attn_applied[0]), 1)
        rnn_input = self.attn_combine(rnn_input).unsqueeze(0)
        rnn_input = F.relu(rnn_input)
    
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        output = F.log_softmax(self.lin_out(output[0]))
        
        return output, hidden[0], attn_weights
    
    def init_rest_hidden(self, input_var):
        if self.n_layers > 1:
            hid_var = Variable(torch.zeros(self.n_layers - 1, 1, self.hidden_size)).cuda()
            res = torch.cat((input_var, hid_var), 0)
            return res
        else:
            return input_var
        
    def mods_split(self):
        mods = list(decoder_rnn.modules())[1:]
        for gru_index, mod in enumerate(mods):
            #print(mod)
            if type(mod) == torch.nn.modules.rnn.GRU:
                break
        return mods[:gru_index], mods[gru_index:]
        
    def mods_attn(self):
        return self.mods_split()[0]
        
    def mods_gru(self):
        return self.mods_split()[1]

decoder_rnn = DecoderRNN(input_size=len(chars_after), hidden_size=tmp_encoder_output.size()[1],
                         chars_encoded_size=tmp_encoder_outputs.size()[1], n_layers=1)
decoder_rnn = decoder_rnn.cuda()
decoder_rnn

tmp = decoder_rnn(Variable(sos_tensor).cuda(), tmp_encoder_output, tmp_encoder_outputs)
#tmp
[v.size() for v in tmp]

DecoderRNN (
  (emb_lin): Linear (33 -> 640)
  (dropout): Dropout (p = 0.1)
  (attn): Linear (1280 -> 30)
  (attn_combine): Linear (1152 -> 640)
  (rnn): GRU(640, 640, batch_first=True)
  (lin_out): Linear (640 -> 33)
)

[torch.Size([1, 33]), torch.Size([1, 640]), torch.Size([1, 30])]

In [29]:
tmp = [{'params': mod.parameters(), 'lr': 3} for mod in decoder_rnn.mods_attn()]
tmp += [{'params': mod.parameters()} for mod in decoder_rnn.mods_gru()]
tmp.append(
    {'params': encoder_rnn.parameters()}
)
tmp

[{'lr': 3, 'params': <generator object Module.parameters at 0x7f066db68150>},
 {'lr': 3, 'params': <generator object Module.parameters at 0x7f066dcf9e08>},
 {'lr': 3, 'params': <generator object Module.parameters at 0x7f066dcf9a40>},
 {'lr': 3, 'params': <generator object Module.parameters at 0x7f066dcf9ba0>},
 {'params': <generator object Module.parameters at 0x7f066dcf9200>},
 {'params': <generator object Module.parameters at 0x7f066dcf99e8>},
 {'params': <generator object Module.parameters at 0x7f066dcf94c0>}]

# Training etc

### Accuracy

In [135]:
test_model_single_sample(None, False, sample=('bef', 'after', '', 'here we go again'.split(' ')))

('bef m tworee e derco',
 'bef m tworee e derco',
 'after',
 ('bef', 'after', '', ['here', 'we', 'go', 'again']))

In [ ]:
def test_model_single_sample(model=None, return_more=False, sample=False):
    decoder_rnn.eval()
    encoder_rnn.eval()
    global encoder_output, encoder_outputs, decoded_output, char_index
    
    if not sample:
        sample = get_random_sample()
    s_bef, s_aft, s_class, s_sentence = sample
        
    words_t = Variable(words_to_word_vectors_tensor(s_sentence, wv_vecs, wv_idx)).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    encoder_output, encoder_outputs = encoder_rnn(words_t, string_t)
    
    decoder_input = Variable(sos_tensor).cuda()
    decoder_hidden = encoder_output
    
    decoded_output = []
    decoder_attns_arr = []
    max_length = 50
    for _ in range(max_length):
        decoder_output, decoder_hidden, decoder_attns = decoder_rnn(decoder_input, decoder_hidden, encoder_outputs)
        decoder_attns_arr.append(decoder_attns)
        #return decoder_output

        topv, topi = decoder_output.data.topk(1)
        char_index = topi[0][0]
        char = chars_after[char_index] # Use own prediction as next input
                
        if char == EOS_TOKEN:
            break

        decoded_output.append(char)
                
        decoder_input = torch.zeros(1, 1, len(chars_after_index))
        decoder_input[0, 0, char_index] = 1
        decoder_input = Variable(decoder_input).cuda()
    
    output = ''.join(decoded_output)
    sample_target = s_aft
    
    decoder_rnn.train()
    encoder_rnn.train()
    
    if return_more:
        return output, decoded_output, decoder_attns_arr, sample
    
    return output, output, sample_target, sample
    
tmp = test_model_single_sample(None)
tmp

In [31]:
def print_local_wrong_predictions(max_results=10):
    arr = get_some_wrong_predictions(None, test_model_single_sample, max_iterations=10000, max_results=max_results)
    for sample, predict, output in arr:
        s_bef, s_aft, s_class, s_sentence = sample
        print("{:<14} => {:<14} || {} \n{:>17} {}".format(s_bef, predict, s_aft, '', s_sentence, ))
print_local_wrong_predictions(2)

27 January 2015 => -mrrfrfrfrfrfrfrfrfr || the twenty seventh of january twenty fifteen 
                  ['<SAMPLE>', '.']
:              => aarfrffrffrffrffrffr || to 
                  ['chemische', 'berichte', '5', '<SAMPLE>', '399', '.']


In [ ]:
%%time
test_model_accuracy(encoder_rnn, test_model_single_sample, n_sample=100)

### Training

In [ ]:
collections.deque(maxlen=100)

In [425]:
def train(s_bef, s_aft, s_sentence, optimizer, loss_function,
          use_teacher_forcing, max_length=20):
    
    global encoder_output, encoder_outputs, decoded_output, char_index
    
    words_t = Variable(words_to_word_vectors_tensor(s_sentence, wv_vecs, wv_idx)).cuda() 
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    target_arr = [chars_after_index[c] for c in list(s_aft)] + [chars_after_index[EOS_TOKEN]]
    optimizer.zero_grad()
    loss = 0
    
    
    encoder_output, encoder_outputs = encoder_rnn(words_t, string_t)
    
    
    decoder_hidden = encoder_output
    decoder_input = Variable(sos_tensor).cuda()

    decoded_output = []
    for i in range(len(target_arr)):
        decoder_output, decoder_hidden, decoder_attns = decoder_rnn(decoder_input, decoder_hidden, encoder_outputs)

        decoder_target_i = target_arr[i]
        decoder_target_i = Variable(torch.LongTensor([decoder_target_i])).cuda()
        loss += loss_function(decoder_output, decoder_target_i)
        
        topv, topi = decoder_output.data.topk(1)
        char_index = topi[0][0]
        char = chars_after[char_index] # Use own prediction as next input
        decoded_output.append(char)
        
        if use_teacher_forcing:
            char_index = target_arr[i] # replace input with right target
        else:
            # use output normally as input 
            if char == EOS_TOKEN:
                break
                
        decoder_input = torch.zeros(1, 1, len(chars_after_index))
        decoder_input[0, 0, char_index] = 1
        decoder_input = Variable(decoder_input).cuda()
        
    if decoded_output[-1] == EOS_TOKEN:
        decoded_output = decoded_output[:-1]

    loss.backward()
    
    # https://github.com/pytorch/examples/blob/master/word_language_model/main.py
    clip_parameters_value = False
    clips = []
    if clip_parameters_value:
        for m in [decoder_rnn, encoder_rnn]:
            clips.append(torch.nn.utils.clip_grad_norm(m.parameters(), clip_parameters_value))
    
    return clips
    
    optimizer.step()

    return ''.join(decoded_output), (loss.data[0] / len(target_arr))


In [432]:
parameters = [p for p in encoder_rnn.parameters()]
#parameters
tmp = [p.grad.max().data[0] for p in parameters]
tmp

[3.73758602142334,
 1.2431634664535522,
 1.3652507066726685,
 0.4760829508304596,
 4.131037712097168,
 1.8706603050231934,
 1.8490597009658813,
 1.8010691404342651,
 2.689138174057007,
 0.7098698616027832,
 4.385306358337402,
 0.7123854160308838,
 3.5336079597473145,
 0.9343348145484924,
 3.8114545345306396,
 0.9602910280227661]

In [430]:
lr = 0.001
tmp = [{'params': mod.parameters(), 'lr': (lr/10)} for mod in decoder_rnn.mods_attn()]
tmp += [{'params': mod.parameters()} for mod in decoder_rnn.mods_gru()]
tmp.append(
    {'params': encoder_rnn.parameters()}
)
optimizer = torch.optim.Adam(tmp, lr=lr)    
loss_function = nn.NLLLoss()
train(s_bef, s_aft, s_sentence, optimizer, loss_function, use_teacher_forcing=False)

[]

In [301]:
optimizer.step()

In [374]:
s_bef

'//www.bidartourism.c'

In [267]:
last_samples[-2]['before']

'//www.bidartourism.com/fort.htmlhttp://www.bidartourism.com/rangeenMahal.htmlhttp://www.bidartourism.com/TarkashMahal.htmlhttp://www.bidartourism.com/GaganMahal.htmlhttp://www.bidartourism.com/Chaubara.htmlhttp://www.holidayiq.com/Solah-Khamba-Mosque-Bidar-Sightseeing-649-757.htmlhttp://www.hoparoundindia.com/karnataka/bidar-attractions/gurudwara-nanak-jhira-saheb.aspx'

In [429]:
tmp = last_samples[-2]
s_bef = tmp['before'][0:5]
s_aft = tmp['after'][0:5]
s_sentence = tmp['sentence'].split(' ')
s_bef

'//www'

In [269]:
words_t = Variable(words_to_word_vectors_tensor(s_sentence, wv_vecs, wv_idx)).cuda() 
string_t = string_to_tensor(s_bef, chars_normal_index)
string_t = Variable(string_t).cuda()
target_arr = [chars_after_index[c] for c in list(s_aft)] + [chars_after_index[EOS_TOKEN]]

In [270]:
encoder_output, encoder_outputs = encoder_rnn(words_t, string_t)

In [273]:
encoder_output[0,0:4]

Variable containing:
-0.3473
-0.5914
 0.6090
 0.1273
[torch.cuda.FloatTensor of size 4 (GPU 0)]

In [274]:
encoder_outputs[0,0:4]

Variable containing:
 0.2169
-0.2463
-0.0235
-0.0335
[torch.cuda.FloatTensor of size 4 (GPU 0)]

In [275]:
encoder_outputs[0,-4:-1]

Variable containing:
 0.4858
 0.1931
 0.7218
[torch.cuda.FloatTensor of size 3 (GPU 0)]

In [276]:
decoder_hidden = encoder_output
decoder_input = Variable(sos_tensor).cuda()

In [277]:
decoder_output, decoder_hidden, decoder_attns = decoder_rnn(decoder_input, decoder_hidden, encoder_outputs)

In [278]:
topv, topi = decoder_output.data.topk(1)
char_index = topi[0][0]
char_index

6

In [35]:
def train_iterations(n_iters=100000, lr=0.001, teacher_forcing_ratio=0.5,
                     print_every=10000, plot_every=1000):
    global optimizer
    start = time.time()
    
    decoder_rnn.train()
    encoder_rnn.train()

    current_loss = 0
    current_loss_iter = 0

    tmp = [{'params': mod.parameters(), 'lr': (lr/10)} for mod in decoder_rnn.mods_attn()]
    tmp += [{'params': mod.parameters()} for mod in decoder_rnn.mods_gru()]
    tmp.append(
        {'params': encoder_rnn.parameters()}
    )
    optimizer = torch.optim.Adam(tmp, lr=lr)
    
    loss_function = nn.NLLLoss()
    
    for iteration in range(1, n_iters + 1):
        model_training.iterations += 1
        
        use_teacher_forcing = random.random() < teacher_forcing_ratio
        
        s_bef, s_aft, s_class, s_sentence = get_random_sample()
        
        result, loss = train(s_bef=s_bef, s_aft=s_aft, s_sentence=s_sentence,
                             optimizer=optimizer,
                             loss_function=nn.NLLLoss(), use_teacher_forcing=use_teacher_forcing,
                             max_length=40 )
        
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            teacher_forcing_str = ""
            if use_teacher_forcing:
                teacher_forcing_str = "(forcing)"
            correct = '✓' if result == s_aft else "✗: {}".format(s_aft)
            
            print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {} -> {} ({}) {}".format(
                      model_training.iterations, iteration/n_iters, time_since(start),
                      current_loss/current_loss_iter, loss,
                      s_bef, result, correct, teacher_forcing_str))

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_training.losses.append(current_loss / plot_every)
            model_training.learning_rates.append(lr)
            current_loss = 0
            current_loss_iter = 0
            
        if model_training.iterations % 50000 == 0 or model_training.iterations == 10:
            model_training.save_models()
            acc = test_model_accuracy(encoder_rnn, test_model_single_sample)
            model_training.accuracy.append(acc)
    
    # test_model_accuracy(model, n_sample=10000)

In [39]:
model_training = ModelTraining(MODEL_SAVE_PATH, [encoder_rnn, decoder_rnn])

Save path: data/models/whole_gen_13_1_only_chars_and_attn


In [40]:
train_iterations(n_iters=50, print_every=9, lr=0.0001)

     9  18% (   0m 0s)   3.449   |   3.37: the -> eee (✗: the) 
Saved model to data/models/whole_gen_13_1_only_chars_and_attn/10_(EncoderRNN/DecoderRNN)
Accuracy: 0.00% (       0/   10000)
    18  36% (  3m 42s)   3.431   |   3.40: revised -> eeeeeee (✗: revised) (forcing)
    27  54% (  3m 42s)   3.412   |   3.35: - -> ee (✗: to) 
    36  72% (  3m 42s)   3.401   |   3.20: after -> eeeee (✗: after) 
    45  90% (  3m 43s)   3.380   |   3.15: 1998 -> eeeeeeeeeeeeeeeeeeeee (✗: nineteen ninety eight) 


In [107]:
decoder_output

Variable containing:

Columns 0 to 12 
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan

Columns 13 to 25 
  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan

Columns 26 to 32 
  nan   nan   nan   nan   nan   nan   nan
[torch.cuda.FloatTensor of size 1x33 (GPU 0)]

In [41]:
train_iterations(n_iters=(1000-model_training.iterations), print_every=500, lr=0.0001)

   550  53% (  0m 16s)   2.962   |   2.91: coastal -> aneeeee (✗: coastal) 


In [42]:
optimizer.param_groups[0]['lr']
optimizer.param_groups[6]['lr']

1e-05

0.0001

In [43]:
train_iterations(n_iters=9000, lr=0.0001, print_every=1000)

  2000  11% (  0m 25s)   2.627   |   2.94: flavour -> aoande (✗: flavor) 
  3000  22% (  0m 55s)   2.436   |   1.15: and -> and (✓) (forcing)
  4000  33% (  1m 21s)   2.131   |   1.75: OS -> s s (✗: o s) (forcing)
  5000  44% (  1m 51s)   1.969   |   1.26: 2009 -> two thennandet ne (✗: two thousand nine) (forcing)
  6000  56% (  2m 19s)   1.827   |   0.00: a -> a (✓) 
  7000  67% (  2m 46s)   1.722   |   1.54: single -> sintte (✗: single) (forcing)
  8000  78% (  3m 14s)   1.633   |   0.00: the -> the (✓) 
  9000  89% (  3m 44s)   1.493   |   0.00: & -> and (✓) 
 10000 100% (  4m 15s)   1.441   |   0.00: & -> and (✓) 


In [44]:
train_iterations(n_iters=90000, print_every=10000)

 20000  11% (  4m 45s)   0.355   |   0.00: st -> saint (✓) 
 30000  22% (  9m 32s)   0.291   |   0.00: Malatov -> malatov (✓) (forcing)
 40000  33% ( 14m 18s)   0.320   |   0.00: a -> a (✓) (forcing)
 50000  44% ( 18m 55s)   0.338   |   0.00: in -> in (✓) 
Saved model to data/models/whole_gen_13_1_only_chars_and_attn/50000_(EncoderRNN/DecoderRNN)
Accuracy: 0.82% (      82/   10000)
 60000  56% (  27m 7s)   0.249   |   0.00: P. -> p (✓) 
 70000  67% ( 31m 49s)   0.262   |   0.00: a -> a (✓) 


IndexError: list index out of range

In [47]:
balanced_data_last_sample['after']

''

In [52]:
def balanced_data_randomize_tmp():
    global balanced_data, balanced_data_length, balanced_data_accessed_counter, balanced_data_randomize_freq
    
    balanced_data = sample_data[sample_data['before'].str.len() == 0]
    balanced_data_randomize_freq = 100000
    
    balanced_data_length = len(balanced_data)
    balanced_data_accessed_counter = 0
balanced_data_randomize = balanced_data_randomize_tmp
balanced_data_randomize()

In [54]:
balanced_data_sample_row()['before']

''

In [82]:
state_dict_path = 'data/models/whole_gen_13_1_only_chars_and_attn/50000_'

decoder_rnn.load_state_dict(torch.load(state_dict_path + 'DecoderRNN'))
encoder_rnn.load_state_dict(torch.load(state_dict_path + 'EncoderRNN'))

In [67]:
train_iterations(n_iters=5, print_every=1, teacher_forcing_ratio=0.5, lr=0.001)

 74198  20% (   0m 0s)   4.645   |   4.64:  -> siu (✗: n a) (forcing)
 74199  40% (   0m 0s)   5.495   |   6.35:  -> sac (✗: n a) 
 74200  60% (   0m 0s)   3.866   |   0.61:  -> s a (✗: n a) (forcing)


IndexError: list index out of range

In [83]:
train_iterations(n_iters=30000, print_every=10000)

 84201  33% (  4m 59s)   0.251   |   0.00: being -> being (✓) (forcing)
 94201  67% (  9m 49s)   0.271   |   0.00: - -> to (✓) (forcing)
Saved model to data/models/whole_gen_13_1_only_chars_and_attn/100000_(EncoderRNN/DecoderRNN)
Accuracy: 0.99% (      99/   10000)
104201 100% ( 17m 55s)   0.289   |   0.00: The -> the (✓) 


In [84]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.001)

114201   5% (  4m 54s)   0.302   |   0.00: ST -> saint (✓) (forcing)
124201  10% (  9m 57s)   0.274   |   0.00: frontal -> frontal (✓) 
134201  15% ( 14m 46s)   0.252   |   0.00: Book -> book (✓) 
144201  20% ( 19m 46s)   0.362   |   0.00: E. -> e (✓) (forcing)
Saved model to data/models/whole_gen_13_1_only_chars_and_attn/150000_(EncoderRNN/DecoderRNN)
Accuracy: 0.73% (      73/   10000)
154201  25% ( 28m 22s)   0.322   |   0.00: vol -> volume (✓) 
164201  30% (  33m 5s)   0.310   |   0.00: & -> and (✓) 
174201  35% (  38m 4s)   0.417   |   0.00: st -> saint (✓) (forcing)
184201  40% ( 42m 43s)   0.350   |   0.00: Magazine -> magazine (✓) 
194201  45% ( 47m 26s)   0.393   |   0.00: In -> in (✓) (forcing)


IndexError: list index out of range

In [87]:
balanced_data_last_sample

sentence_id                                               687600
token_id                                                       7
class                                                      PLAIN
before                                                         -
after                                                         to
class_org                                                  PLAIN
a_word_ind                                               [57, 0]
sentence       the grand opera house was constructed 1898 <SA...
Name: 601296, dtype: object

In [128]:
state_dict_path = 'data/models/whole_gen_13_1_only_chars_and_attn/100000_'

decoder_rnn.load_state_dict(torch.load(state_dict_path + 'DecoderRNN'))
encoder_rnn.load_state_dict(torch.load(state_dict_path + 'EncoderRNN'))

In [132]:
train_iterations(n_iters=100000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.001)

Saved model to data/models/whole_gen_13_1_only_chars_and_attn/200000_(EncoderRNN/DecoderRNN)
Accuracy: 0.79% (      79/   10000)
205635  10% (  8m 41s)   0.290   |   0.00: No -> number (✓) 
215635  20% ( 13m 39s)   0.289   |   0.46: 28,200 -> twenty eighthohousand swe hundred (✗: twenty eight thousand two hundred) (forcing)
225635  30% ( 18m 42s)   0.271   |   0.00: currently -> currently (✓) 
235635  40% ( 23m 22s)   0.311   |   0.00: equip -> equip (✓) 
245635  50% ( 28m 18s)   0.335   |   0.00: On -> on (✓) 
Saved model to data/models/whole_gen_13_1_only_chars_and_attn/250000_(EncoderRNN/DecoderRNN)
Accuracy: 0.82% (      82/   10000)


KeyboardInterrupt: 

In [ ]:
Updated target_arr to include eos

In [133]:
print_local_wrong_predictions()

Hell           => helleratelllerseed e || hell 
                  ['he', 'also', 'played', 'drums', 'for', 'both', 'wax', 'mannequin', 'and', 'the', 'burning', '<SAMPLE>', 'on', 'the', 'hear', 'some', 'evil', 'tour', '.']
at             => att hundred three th || at 
                  ['he', 'also', 'worked', '<SAMPLE>', 'the', 'local', 'red', 'owl', 'supermarket', 'alongside', 'his', 'brother', '.']
of             => ofe rscoreeterared f || of 
                  ['"', 'severe', 'hepatotoxicity', 'following', 'ingestion', '<SAMPLE>', 'herbalife', 'nutritional', 'supplements', 'contaminated', 'with', 'bacillus', 'subtilis', '"', '.']
959            => nine hundred fifty n || nine hundred fifty nine 
                  ['"', '1987', '/', '88', 'porsche', '<SAMPLE>', '"', '.']
the            => the r c hee hundred  || the 
                  ['security', 'forces', 'subsequently', 'reach', 'the', 'area', 'and', 'begin', 'probing', '<SAMPLE>', 'circumstances', 'surrounding', 'the', 'attack', 

In [143]:
train_iterations(n_iters=50000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.001)

260467  20% (   5m 0s)   0.487   |   0.00: wounded -> wounded (✓) (forcing)
270467  40% (  10m 0s)   0.566   |   3.51: www.visionzeroboston.org/Orlov -> w w w i coi oos o o o o o coo o o o o o o o o o o o coo o o o o o o o co (✗: w w w dot v i s i o n z e r o b o s t o n dot o r g s l a s h o r l o v) 


IndexError: list index out of range

In [180]:
char_index

4433358438331329401

In [ ]:
z

In [128]:
state_dict_path = 'data/models/whole_gen_13_1_only_chars_and_attn/100000_'

decoder_rnn.load_state_dict(torch.load(state_dict_path + 'DecoderRNN'))
encoder_rnn.load_state_dict(torch.load(state_dict_path + 'EncoderRNN'))

In [132]:
train_iterations(n_iters=100000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.001)

Saved model to data/models/whole_gen_13_1_only_chars_and_attn/200000_(EncoderRNN/DecoderRNN)
Accuracy: 0.79% (      79/   10000)
205635  10% (  8m 41s)   0.290   |   0.00: No -> number (✓) 
215635  20% ( 13m 39s)   0.289   |   0.46: 28,200 -> twenty eighthohousand swe hundred (✗: twenty eight thousand two hundred) (forcing)
225635  30% ( 18m 42s)   0.271   |   0.00: currently -> currently (✓) 
235635  40% ( 23m 22s)   0.311   |   0.00: equip -> equip (✓) 
245635  50% ( 28m 18s)   0.335   |   0.00: On -> on (✓) 
Saved model to data/models/whole_gen_13_1_only_chars_and_attn/250000_(EncoderRNN/DecoderRNN)
Accuracy: 0.82% (      82/   10000)


KeyboardInterrupt: 

In [200]:
state_dict_path = 'data/models/whole_gen_13_1_only_chars_and_attn/250000_'

decoder_rnn.load_state_dict(torch.load(state_dict_path + 'DecoderRNN'))
encoder_rnn.load_state_dict(torch.load(state_dict_path + 'EncoderRNN'))

In [242]:
train_iterations(n_iters=100000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.001)

285054  10% (  5m 20s)   0.439   |   0.00: ltd -> limited (✓) (forcing)
295054  20% ( 10m 23s)   0.460   |   0.00: fire -> fire (✓) 
Saved model to data/models/whole_gen_13_1_only_chars_and_attn/300000_(EncoderRNN/DecoderRNN)
Accuracy: 77.47% (    7747/   10000)
305054  30% ( 16m 48s)   0.597   |   1.10: 1967 -> nineteee sex<EOS>p seven (✗: nineteen sixty seven) (forcing)


IndexError: list index out of range

In [266]:
state_dict_path = 'data/models/whole_gen_13_1_only_chars_and_attn/300000_'

decoder_rnn.load_state_dict(torch.load(state_dict_path + 'DecoderRNN'))
encoder_rnn.load_state_dict(torch.load(state_dict_path + 'EncoderRNN'))

In [ ]:
train_iterations(n_iters=100000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=100000, print_every=10000, teacher_forcing_ratio=0.1, lr=0.001)

In [ ]:
print_local_wrong_predictions()

In [148]:
last_samples[-2]['sentence']

'retrieved june 17 , 2012 from <SAMPLE> noise on imdb miller .'

In [ ]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0.1, lr=0.0005)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.1, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0, lr=0.0005)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0.2, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=400000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
train_iterations(n_iters=10000, lr=0.001, print_every=1000)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0.3, lr=0.001)

In [ ]:
state_dict_path = 'data/models/whole_gen_13_only_chars_and_attn/10_'
#state_dict_path = 'data/models/whole_gen_13_only_chars_and_attn/250000_'

decoder_rnn.load_state_dict(torch.load(state_dict_path + 'DecoderRNN'))
encoder_rnn.load_state_dict(torch.load(state_dict_path + 'EncoderRNN'))

### Show attention

In [ ]:
balanced_data.sample(5)

In [ ]:
balanced_data.groupby('class')['class'].count()
len(balanced_data)

In [ ]:
def balanced_data_randomize_long():
    global balanced_data, balanced_data_length, balanced_data_accessed_counter, balanced_data_randomize_freq
    
    bal_data = pd.concat([v.sample(min(2000, len(v))) for k, v in balanced_data_classes_select])
    long_data = sample_data[sample_data['before'].str.len()>8].sample(4000)
    elec_data = sample_data[sample_data['class']=='ELECTRONIC']
    let_long_data = sample_data[(sample_data['class'] == 'LETTERS') & (sample_data['before'].str.len() > 5)]
    balanced_data = pd.concat([bal_data, long_data, elec_data, let_long_data])#.drop_duplicates()
    balanced_data = balanced_data[~balanced_data.index.duplicated(keep='first')]
    
    balanced_data_length = len(balanced_data)
    balanced_data_randomize_freq = balanced_data_length * 0.5
    balanced_data_accessed_counter = 0

balanced_data_randomize = balanced_data_randomize_long
balanced_data_randomize()

In [ ]:
def debug_show_attention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure(figsize=(15,6))
    ax = fig.add_subplot(111)
    #cax = ax.matshow(attentions.numpy(), cmap='bone')
    cax = ax.matshow(attentions, cmap='bone')
    fig.colorbar(cax)
    # Set up axes
    #ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
    #ax.set_xticklabels([''] + input_sentence + ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)
    
    input_sentence = input_sentence + ['<EOS>']
    #inp_arr = ["{}\n{}".format(input_sentence[i], input_sentence[-1-i]) for i in range(len(input_sentence))]
    inp_arr = input_sentence
    ax.set_xticklabels([''] + inp_arr, rotation=0)

    # Show label at every tick
    ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(1))

    plt.show()
    
def debug_eval_sample_show_attention():
    
    sample_row = balanced_data_sample_row()
    #sample_row = balanced_data[balanced_data['before'].str.len()>15].sample(1).iloc[0]
    sample = sample_row['before'], sample_row['a_word_ind'], sample_row['class'], sample_row['sentence'].split(' ')

    output, decoded_output, decoder_attns_arr, sample = test_model_single_sample(None, 
                                                            return_more=True, sample=sample)
    print('input:  ', sample[0])
    print('output: ', decoded_output)
    print('target:   ', ' '.join([words_after_common[w] for w in sample[1][:-1]]))

    attns = np.array([arr.data[0].cpu().numpy() for arr in decoder_attns_arr])

    debug_show_attention(list(sample[0]), decoded_output, attns)
    #plt.matshow(attns)

debug_eval_sample_show_attention()

In [ ]:
balanced_data.groupby('class')['class'].unique()

In [ ]:
def test_in_categories(iter_len = 1000):
    wrong_preds = {}
    for cat in categories_all:
        tmp_data = sample_data[sample_data['class'] == cat].sample(iter_len)
        correct_n = 0
        wrong_preds_arr = []

        for _ in range(iter_len):
            sample_row = tmp_data.iloc[_]
            sample = sample_row['before'], sample_row['a_word_ind'], sample_row['class'], sample_row['sentence']

            output, t1, sample_target, t2 = test_model_single_sample(None, sample=sample)
            if output == sample_target:
                correct_n += 1
            else:
                wrong_preds_arr.append([sample_target, output])

        print("{:>10}: {:>5d}/{:>5d} ({:>4.0%})".format(cat, correct_n, iter_len, correct_n/iter_len))
        wrong_preds[cat] = wrong_preds_arr
    return wrong_preds

In [ ]:
wrong_preds = test_in_categories(3000)

In [ ]:
wrong_preds['LETTERS']

In [ ]:
# With training longer words
wrong_preds = test_in_categories()